# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [112]:
import pandas as pd # импорт библиотеки pandas
from pymystem3 import Mystem # импорт библиотеки pymystem3
m = Mystem() # присваиваем переменной m, функцию Mystem()
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df

In [113]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Наблюдения: 
1. В названиях колонок не обнаружены нарушения стиля.
2. В колонках `days_employed` и `total_income` обнаружены пропуски данных.

In [114]:
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Обнаружены проблемы с данными:
- В колонке `days_employed` есть отрицательные значения (стаж не может быть отрицательным) и не реалистичные значения.

**Просмотрим уникальные значения и их кол-во в интересующих нас столбцах:**

In [115]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружены неверные значения:
- 20 - слишком большое значение для кол-ва детей, скорее всего ошибочно добавлен 0, нужно исправить на 2
- -1 - кол-во детей не может быть отрицательным, скорее всего ошибочно поставлен знак "-", нужно исправить на 1 

In [116]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

- пропусков и неверных значений не обнаружено.
- одни и те же значения указаны в разном формате ('высшее' и 'ВЫСШЕЕ', 'среднее' и 'СРЕДНЕЕ' и т.д.)

In [117]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

- пропусков и неверных значений не обнаружено
- кол-во различных значений индетификатора, не совпадает с кол-вом различных значений наименований уровня образования

In [118]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

- пропусков и неверных значений не обнаружено

In [119]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

- пропусков и неверных значений не обнаружено.
- кол-во различных значений индетификатора, совпадает с кол-вом различных значений наименований семейного положения

In [120]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

- обнаружено одно неверное значение

In [121]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

- пропусков и неверных значений не обнаружено.

In [122]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

- пропусков и неверных значений не обнаружено.

In [123]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

- Обнаружены похожие значения, необходимо провести лемматизацию и категорезировать данные

**Вывод**

В каждой строке таблицы — данные о клиетах банка.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски, дубликаты и ошибки в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Обработка пропусков

В столбце `days_employed` есть некорректные данные: отрицательные значения, не реалистичные значения и пропуски. Возможно данные повреждены или указаны в неверном формате. Данный столбец не нужен нам для дальнейшего исследования, поэтому заменим пропуски нулями, а остальные значения в этом столбце, оставим без изменений.

In [124]:
df['days_employed'] = df['days_employed'].fillna(0) # замена пропусков на нули в стобце 'days_employed'
df['days_employed'].isna().sum() #проверяем что все пропуски удалены

0

 В столбце `total_income` были обнаружены пропуски данных, займемся их устранением.

In [125]:
 # проверяем,нет ли отрицательных значений в столбце, так как доход должен быть положительным значением
df[df['total_income'] <= 0]['total_income'].count()

0

Отрицательных значений не обнаружено

In [126]:
# находим средние арефметическое значение в стоблце
df_total_income_mean = df['total_income'].mean()
df_total_income_mean

167422.30220817294

In [127]:
# находим медианное значение в столбце
df_total_income_median = df['total_income'].median()
df_total_income_median

145017.93753253992

In [128]:
# КОД РЕВЬЮЕРА

df['total_income'].describe().loc[['mean', '50%']]

mean    167422.302208
50%     145017.937533
Name: total_income, dtype: float64

Средние арифметическое и медианное значение, отличаются довольно значительно, поэтому заменим пропуски на медианное значение. Перед заменой пропусков, создадим новый столбец 'total_income_replace', в котором ‘0’ – замена значения в столбце 'total_income' в данной строке не проводилась, а ‘1’ -  замена значения производилась. Данный столбец создаем для того чтобы  в дальнейшем можно было удалить строки в которых проводилась замена, так как они могут влиять на результаты исследования: “ Есть ли зависимость между уровнем дохода и возвратом кредита в срок?”. Сразу не удаляем данные строки, так как в них могут содержаться полезные данные для других исследований.  

In [129]:
df[df['dob_years'] <= 20]['dob_years'].value_counts()

0     101
20     51
19     14
Name: dob_years, dtype: int64

In [130]:
pd.qcut(df[df['dob_years'] >= 19]['dob_years'], q=5)

0          (39.0, 47.0]
1          (32.0, 39.0]
2          (32.0, 39.0]
3        (18.999, 32.0]
4          (47.0, 56.0]
              ...      
21520      (39.0, 47.0]
21521      (56.0, 75.0]
21522      (32.0, 39.0]
21523      (32.0, 39.0]
21524      (39.0, 47.0]
Name: dob_years, Length: 21424, dtype: category
Categories (5, interval[float64]): [(18.999, 32.0] < (32.0, 39.0] < (39.0, 47.0] < (47.0, 56.0] < (56.0, 75.0]]

In [131]:
df['total_income_replace'] = 0 # создаём новый столбец, в котором все значения нули
df['total_income'] = df['total_income'].fillna(-10) # заменяем все пропуски в столбце на значение "-10"
df.loc[df['total_income'] == -10, 'total_income_replace'] = 1 # устаналиваем метку в столбце total_income_replace'
#df.loc[df['total_income'] == -10, 'total_income'] = df_total_income_median # меняем в столбце значение "-10" на медианное значение
#df['total_income'].isna().sum() #проверяем что все пропуски удалены

В столбце 'gender' было обнаружено одно неверное значение, удаление одной строки не повлияет на ваборку, поэтуму удаляем её.

In [132]:
df= df[df['gender'] !='XNA'] # удаляем строку за значением 'XNA'
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

**Вывод**

Все пропуски удалены, можем двигаться дальше.

### Замена типа данных

Производем замену типа данных с float на int, в стобце 'total_income' с помощью функции astype. Функция to_numeric не подходит так как она переводит значения, только в тип float.

In [133]:
df['total_income'] = df['total_income'].astype('int') 
df.info() #проверяем что тип столбца изменился на int

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
children                21524 non-null int64
days_employed           21524 non-null float64
dob_years               21524 non-null int64
education               21524 non-null object
education_id            21524 non-null int64
family_status           21524 non-null object
family_status_id        21524 non-null int64
gender                  21524 non-null object
income_type             21524 non-null object
debt                    21524 non-null int64
total_income            21524 non-null int64
purpose                 21524 non-null object
total_income_replace    21524 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.3+ MB


**Вывод**

В столбце `total_income` произведена замена вещественного типа данных на целочисленный. В столбце `days_employed` не меняем тип данных так как он нам не нужен для дальнейших исследований. Можно двигаться дальше. 

### Обработка дубликатов

Заменяем значения -1 на 1 и 20 на 2, в столбце `children`

In [134]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts() # делаем проверку, что замены прошли успешно

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Меняем прописной регистр на строчный в столбце `education`

In [135]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()# делаем проверку, что замена регистра прошла успешно

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Дубликаты удалены, можно двигаться дальше.

### Дополнение 1

Заменяем пропуски дохода медианой, взятой по следующим характеристикам клиента: пол + образование + тип занятости.

In [136]:
# Создаем сводную таблицу по столбцам: 'gender'(пол), 'education'(образование), 'income_type'(тип занятости)
# считаем медианное значение по столбцу 'total_income' (суммарный заработок)
df1 = pd.pivot_table(df, values='total_income', index=['gender', 'education', 'income_type'], aggfunc='median').reset_index()
# Создаем новый датафрейм, в него записывыем все строки с пометкой в столбце 'total_income_replace'
df2 = df[df['total_income_replace'] == 1]
# Удаляем столбец 'total_income' в новом датафрейме df2
df2.pop('total_income')
# Сбрасываем значение индекса в новом датафрейме df2
df2 = df2.reset_index()
# объединяем таблицы df2 и df1
df2 = df2.merge(df1, on=['gender', 'education', 'income_type'])
# Удаляем все строки с пометкой в столбце 'total_income_replace'
df = df[df['total_income_replace'] != 1]
# Сбрасываем значение индекса
df = df.reset_index()
# Объединяем таблицы df и df2
df = pd.concat([df, df2], ignore_index=True, sort=False)
#проверяем что все пропуски удалены
df['total_income'].isna().sum() 

0

**Вывод**

Все пропуски удалены, можем двигаться дальше.

### Лемматизация

Создадим список лемматизированных значений столбца `purpose`

In [137]:
lemmas = [] # создаем пустой список в который будет добавлять лемматизированные значения
purpose_list = df['purpose'].unique() # создаем список уникальных значений столбца 'purpose'

# Запускаем цикл, который лемматизирует слова из списка purpose_list
for i in purpose_list:
    lemma = ''.join(m.lemmatize(i)) # лемматизируем слова и объединяем их в одну строку (для лучшей наглядности)
    lemmas.append(lemma) # добавляем лемматизированное значение в список lemmas                  
lemmas

['покупка жилье\n',
 'приобретение автомобиль\n',
 'дополнительный образование\n',
 'сыграть свадьба\n',
 'операция с жилье\n',
 'образование\n',
 'на проведение свадьба\n',
 'покупка жилье для семья\n',
 'покупка недвижимость\n',
 'покупка коммерческий недвижимость\n',
 'покупка жилой недвижимость\n',
 'строительство собственный недвижимость\n',
 'недвижимость\n',
 'строительство недвижимость\n',
 'на покупка подержать автомобиль\n',
 'на покупка свой автомобиль\n',
 'операция с коммерческий недвижимость\n',
 'жилье\n',
 'операция со свой недвижимость\n',
 'автомобиль\n',
 'заниматься образование\n',
 'получение образование\n',
 'сделка с подержанный автомобиль\n',
 'автомобиль\n',
 'свадьба\n',
 'получение дополнительный образование\n',
 'покупка свой жилье\n',
 'операция с недвижимость\n',
 'получение высокий образование\n',
 'свой автомобиль\n',
 'сделка с автомобиль\n',
 'профильный образование\n',
 'высокий образование\n',
 'покупка жилье для сдача\n',
 'на покупка автомобиль\n',

**Вывод**

Лемматизировали столбец `purpose`.
Проанализировав полученный список можно выделить четыре категории целей получения кредита:
 - недвижимость
 - автомобиль
 - свадьба
 - образование

### Категоризация данных

Создаем функцию purpose_categories, которая будет осуществлять категоризацию столбца `purpose`

In [138]:
def purpose_categories(purpose):
    
    lemma_purpose = ''.join(m.lemmatize(purpose)) # лемматизируем слова и объединяем их в одну строку
    
    # если в лемматизированной строке встречается слово "авто", то возвращаем категорию "автомобиль"
    if 'авто' in lemma_purpose:
        return 'автомобиль'
    
    # если в лемматизированной строке встречается слово "жилье" или "недвижимость", то возвращаем категорию "недвижимость"
    if ('жилье' in lemma_purpose) or ('недвижимость' in lemma_purpose):
        return 'недвижимость'
    
    # если в лемматизированной строке встречается слово "свадьба", то возвращаем категорию "свадьба"
    if 'свадьба' in lemma_purpose:
        return 'свадьба'
    
    # если в лемматизированной строке встречается слово "образование", то возвращаем категорию "образование"
    if 'образование' in lemma_purpose:
        return 'образование'
    
    # если в лемматизированной строке ни одна из заданных категорий не найдена, то возвращаем значение "категория не обнаружена"
    return 'категория не обнаружена'

In [139]:
# Создаем новый столбец 'purpose_categories, в которой записываем категории столбца 'purpose'.
# С помощь функции apply, применяем созданную нами функцию purpose_categories к столбцу 'purpose'
df['purpose_categories'] = df['purpose'].apply(purpose_categories)
df['purpose_categories'].value_counts() # проверяем, что категории создались верно

недвижимость    10839
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_categories, dtype: int64

In [140]:
df_income = df.copy() # создаем копию датафрейма df
#удаляем строки в которых есть пометка в столбце 'total_income_replace'
df_income = df_income[df_income['total_income_replace'] != 1] 
df_income.pop('total_income_replace') # удаляем столбец total_income_replace'  
df_income.info() # проверяем что помеченные строки и столбец удалены

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19350 entries, 0 to 19349
Data columns (total 14 columns):
index                 19350 non-null int64
children              19350 non-null int64
days_employed         19350 non-null float64
dob_years             19350 non-null int64
education             19350 non-null object
education_id          19350 non-null int64
family_status         19350 non-null object
family_status_id      19350 non-null int64
gender                19350 non-null object
income_type           19350 non-null object
debt                  19350 non-null int64
total_income          19350 non-null float64
purpose               19350 non-null object
purpose_categories    19350 non-null object
dtypes: float64(2), int64(6), object(6)
memory usage: 2.2+ MB


Создаем функцию income_categorie, которая будет осуществлять категоризацию столбца `income`

In [141]:
def income_categories(income):
    
    # если суммарный доход меньше или равен 100 тыс., то возвращаем категорию "до 100 тыс."
    if income <= 100000:
        return 'до 100 тыс.'
    
    # если суммарный доход больше 100 тыс., но меньше или равен 150 тыс., то возвращаем категорию "от 100 до 150 тыс."
    if income <= 150000:
        return 'от 100 до 150 тыс.'
    
    # если суммарный доход больше 150 тыс., но меньше или равен 200 тыс., то возвращаем категорию "от 150 до 200 тыс."
    if income <= 200000:
        return 'от 150 до 200 тыс.'
    
    # во всех остальных случаях возвращаем категорию "более 200 тыс."
    return 'более 200 тыс.'

In [142]:
# Создаем новый столбец 'total_income_categories, в которой записываем категории столбца 'total_income'.
# С помощь функции apply, применяем созданную нами функцию income_categories к столбцу 'total_income'
df_income['total_income_categories'] = df_income['total_income'].apply(income_categories)
df_income['total_income_categories'].value_counts() # проверяем, что категории создались верно

от 100 до 150 тыс.    5704
более 200 тыс.        5065
до 100 тыс.           4463
от 150 до 200 тыс.    4118
Name: total_income_categories, dtype: int64

**Вывод**

Проведена категоризация значений, столбцов `purpose` и `total_income`.
Предабработка данных закончена можно переходить к исследованиям.

In [145]:
# Создаем словарь для пары столбцов `education` и `education_id`
education_dict = df[['education_id', 'education']]
# Удаляем дублирующийся значения
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [146]:
# Создаем словарь для пары столбцов `family_status` и `family_status`
family_status_dict = df[['family_status_id', 'family_status']]
# Удаляем дублирующийся значения
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [147]:
# Считаем кол-во клиентов с детьми и без детей
children_yes_count = df[df['children'] > 0]['children'].count()
children_no_count = df[df['children'] == 0]['children'].count()

# Считаем кол-во клиентов с детьми и без детей, которые имели задолжность по возврату кредитов
children_yes_debt = df[df['children'] > 0]['debt'].sum()
children_no_debt = df[df['children'] == 0]['debt'].sum()

# Считаем и выводим на экран процент клиентов с детьми, которые имели задолженность по возврату кредитов
try:
    print(f'Процент клиентов с детьми, которые имели задолженность по возврату кредитов: {children_yes_debt / children_yes_count:.2%}')
except:
    print('Количество клиентов с детьми равно 0')

# Считаеми выводим на экран процент клиентов без детей, которые имели задолженность по возврату кредитов
try:
    print(f'Процент клиентов без детей, которые имели задолженность по возврату кредитов: {children_no_debt / children_no_count:.2%}')
except:
    print('Количество клиентов без детей равно 0')    

Процент клиентов с детьми, которые имели задолженность по возврату кредитов: 9.19%
Процент клиентов без детей, которые имели задолженность по возврату кредитов: 7.51%


In [148]:
# Считаем кол-во клиентов: с одним ребенком, двумя детьми и более чем двумя детьми
children_one_count = df[df['children'] == 1]['children'].count()
children_two_count = df[df['children'] == 2]['children'].count()
children_more_two_count = df[df['children'] > 2]['children'].count()

# Считаем кол-во клиентов: с одним ребенком, двумя детьми и более чем двумя детьми, которые имели задолжность по возврату кредитов
children_one_debt = df[df['children'] == 1]['debt'].sum()
children_two_debt = df[df['children'] == 2]['debt'].sum()
children_more_two_debt = df[df['children'] > 2]['debt'].sum()

# Считаем и выводим на экран процент клиентов с одним ребенком, которые имели задолженность по возврату кредитов
try:
    print(f'Процент клиентов с одним ребенком, которые имели задолженность по возврату кредитов: {children_one_debt / children_one_count:.2%}')
except:
    print('Количество клиентов с одним ребенком равно 0')
    
# Считаем и выводим на экран процент клиентов с двумя детьми, которые имели задолженность по возврату кредитов
try:
    print(f'Процент клиентов с двумя детьми, которые имели задолженность по возврату кредитов: {children_two_debt / children_two_count:.2%}')
except:
    print('Количество клиентов с двумя детьми равно 0')

# Считаем и выводим на экран процент клиентов с более чем двумя детьми, которые имели задолженность по возврату кредитов
try:
    print(f'Процент клиентов с более чем двумя детьми, которые имели задолженность по возврату кредитов: {children_more_two_debt / children_more_two_count:.2%}')
except:
    print('Количество клиентов с более чем двумя детьми равно 0')

Процент клиентов с одним ребенком, которые имели задолженность по возврату кредитов: 9.15%
Процент клиентов с двумя детьми, которые имели задолженность по возврату кредитов: 9.48%
Процент клиентов с более чем двумя детьми, которые имели задолженность по возврату кредитов: 8.16%


**Вывод**

По итогам исследования, можно сформировать гипотезу что клиенты без детей - реже допускают просрочки по возврату кредитов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [152]:
# Создаем список с уникальными значениями столбца 'family_status'
family_status = df['family_status'].unique()

# Запускаем цикл, который проходится по всем уникальными значениями столбца 'family_status'
for i in family_status:
    # Считаем кол-во клиентов в категории
    family_status_count = df[df['family_status'] == i]['family_status'].count()
    # Считаем кол-во клиентов в категории, которые имели задолжность по кредиту
    family_status_debt = df[df['family_status'] == i]['debt'].sum()
    # Считаем и выводим на экран процент клиентов в категории, которые имели задолженность по возврату кредитов
    print (f'Процент клиентов с семейным положением "{i}", которые имели задолженность по возврату кредитов: {family_status_debt / family_status_count:.2%}')

Процент клиентов с семейным положением "женат / замужем", которые имели задолженность по возврату кредитов: 7.52%
Процент клиентов с семейным положением "гражданский брак", которые имели задолженность по возврату кредитов: 9.29%
Процент клиентов с семейным положением "вдовец / вдова", которые имели задолженность по возврату кредитов: 6.56%
Процент клиентов с семейным положением "в разводе", которые имели задолженность по возврату кредитов: 7.11%
Процент клиентов с семейным положением "Не женат / не замужем", которые имели задолженность по возврату кредитов: 9.74%


**Вывод**

По итогам исследования, можно сформировать гипотезу, что клиенты с семейным положением: "женат / замужем", "вдовец / вдова" и "в разводе", реже допускают просрочки по возврату кредитов, чем клиенты с семейным положением: "гражданский брак" и "Не женат / не замужем".  

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [153]:
# Создаем список с категориями - суммарного заробтка клиентов
income_categories = ['до 100 тыс.', 'от 100 до 150 тыс.', 'от 150 до 200 тыс.', 'более 200 тыс.']

# Запускаем цикл, который проходится по всем категориями
for i in income_categories:
    # Считаем кол-во клиентов в категории
    income_categories_count = df_income[df_income['total_income_categories'] == i]['total_income_categories'].count()
    # Считаем кол-во клиентов в категории, которые имели задолжность по кредиту
    income_categories_debt = df_income[df_income['total_income_categories'] == i]['debt'].sum()
    # Считаем и выводим на экран процент клиентов в категории, которые имели задолженность по возврату кредитов
    print (f'Процент клиентов с уровнем дохода {i}, которые имели задолженность по возврату кредитов: {income_categories_debt / income_categories_count:.2%}')

Процент клиентов с уровнем дохода до 100 тыс., которые имели задолженность по возврату кредитов: 7.93%
Процент клиентов с уровнем дохода от 100 до 150 тыс., которые имели задолженность по возврату кредитов: 8.61%
Процент клиентов с уровнем дохода от 150 до 200 тыс., которые имели задолженность по возврату кредитов: 8.94%
Процент клиентов с уровнем дохода более 200 тыс., которые имели задолженность по возврату кредитов: 7.07%


**Вывод**

По итогам исследования, можно сформировать гипотезу, что клиенты с доходом более 200 тыс., реже всего допускают просрочки по возврату кредитов, а клиенты с доходом от 100 до 200 тыс., чаще всего.  

- Как разные цели кредита влияют на его возврат в срок?

In [154]:
# Создаем список с уникальными значениями столбца 'purpose_categories'
purpose_categories = df['purpose_categories'].unique()

# Запускаем цикл, который проходится по всем категориями
for i in purpose_categories:
    # Считаем кол-во клиентов в категории
    purpose_categories_count = df[df['purpose_categories'] == i]['purpose_categories'].count()
    # Считаем кол-во клиентов в категории, которые имели задолжность по кредиту
    purpose_categories_debt = df[df['purpose_categories'] == i]['debt'].sum()
    # Считаем и выводим на экран процент клиентов в категории, которые имели задолженность по возврату кредитов
    print (f'Процент клиентов с целью кредита "{i}", которые имели задолженность по возврату кредитов: {purpose_categories_debt / purpose_categories_count:.2%}')

Процент клиентов с целью кредита "недвижимость", которые имели задолженность по возврату кредитов: 7.21%
Процент клиентов с целью кредита "автомобиль", которые имели задолженность по возврату кредитов: 9.34%
Процент клиентов с целью кредита "образование", которые имели задолженность по возврату кредитов: 9.20%
Процент клиентов с целью кредита "свадьба", которые имели задолженность по возврату кредитов: 7.92%


**Вывод**
По итогам исследования, можно сформировать гипотезу, что клиенты берущие кредит с целью: "недвижимость" и "свадьба", реже допускают просрочки по возврату кредитов, чем клиенты берущие кредит с целью: "автомобиль" и "образование".  

## Общий вывод

По итогам исследований сформированы следующие гипотезы:

- клиенты без детей - реже допускают просрочки по возврату кредитов: 7.51%, против 9.19% у клиентов с детьми 
- клиенты с семейным положением: "женат / замужем", "вдовец / вдова" и "в разводе", реже допускают просрочки по возврату кредитов (6,56 - 7,59%), чем клиенты с семейным положением: "гражданский брак" и "Не женат / не замужем" (9,29 - 9,74%). 
- клиенты с доходом более 200 тыс., реже всего допускают просрочки по возврату кредитов (7,07 - 7,93%), а клиенты с доходом от 100 до 200 тыс., чаще всего (8,61 - 8,94%)
- клиенты берущие кредит с целью: "недвижимость" и "свадьба", реже допускают просрочки по возврату кредитов (7,21 - 7,92%), чем клиенты берущие кредит с целью: "автомобиль" и "образование (9,2 - 9,34%)

Для подтверждения данных гипотезы нужно проводить более глубокое исследование, основанное на различных факторах, влияющих на возврат клиентами кредитов без задолженностей.